# **관악구 동네 이야기**
1. 데이터 불러오기
1. 주소를 바탕으로 위도, 경도 데이터 가져오기
1. Naver 지도 위에서 해당 내용 Marker (in React.js)

## **1 데이터 전처리 작업**
데이터를 불러온 뒤, 처리에 용이한 Type 으로 변환하기

In [1]:
# 데이터 CSV 불러오기
import pandas as pd
file_name = "./data/ourtown_store.xlsx"
data_xls = pd.read_excel(file_name, header=1)
data_columns = data_xls.columns.to_list()
data_columns.pop()
data_xls.columns = data_columns + ['기타']
data_xls.head(3)
data_xls[data_xls["연번"].isnull()]

,연번,상호명,점주명,연락처,주소,업종,행정동,사진,임대차,비고,기타
44,NaN,NaN,NaN,NaN,"호암로24길 1, 2ㆍ3층",NaN,NaN,O,O,NaN,NaN


In [2]:
# Filling the NaN data
for _ in data_xls.columns.to_list()[:-1]:
    data_xls[_] = data_xls[_].fillna(method="ffill")

# Other field filled with blank text
data_xls["기타"] = data_xls["기타"].fillna('')

# Checking the NaN data
data_xls[data_xls["연번"].isnull()]

,연번,상호명,점주명,연락처,주소,업종,행정동,사진,임대차,비고,기타


## **2 위도, 경도 데이터 생성하기**
주소 데이터를 활용하여 위도/ 경도 데이터 생성하기

In [3]:
# data_xls.to_csv(file_name.replace("xlsx","csv"), index=None)
print(data_xls['주소'][0])
data_xls.head(3)

은천로 28, 봉일프라자 나-120


,연번,상호명,점주명,연락처,주소,업종,행정동,사진,임대차,비고,기타
0,1.0,모든홍삼,주옥자,010-7908-0324,"은천로 28, 봉일프라자 나-120",기타판매,은천동,X,X,개별점포,
1,2.0,종로떡집,김기철,010-4606-7280,국회단지길 21,떡집,은천동,X,X,개별점포,
2,3.0,김원화 미용실,김원화,010-3778-1652,국회단지길 35,미용업,보라매동,X,X,개별점포,


In [4]:
data_xls['주소']

0      은천로 28, 봉일프라자 나-120
1                 국회단지길 21
2                 국회단지길 35
3                봉천로27길 10
4               양녕로 16, 1층
              ...         
146            남부순환로247길 3
147          난곡로24길 29, 1층
148             신림로65길 31 
149           남부순환로214길 29
150              난곡로24길 32
Name: 주소, Length: 151, dtype: object

In [5]:
# 건물관리번호 : 1144012700115950000000001
#     X좌표 : 945959.0381341814
#         Y좌표 : 1953851.7348996028
#             건물명 : 한국지역정보개발원(KLID Tower)